In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.image as img
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models


In [ ]:
NUM_CLASSES = 4
IMAGE_RESIZE = 256
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 15
EARLY_STOP_PATIENCE = 6

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 16
BATCH_SIZE_VALIDATION = 16

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [ ]:
### 
### Below systax is available with TensorFlow 1.11 onwards but this upgrade is not available for Kaggle kernel yet
###
import tensorflow as tf
print(tf.__version__)
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

2.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 256

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence

data_generator = ImageDataGenerator(rescale = 1./255,
                                    validation_split = 0.2,
                                  preprocessing_function=preprocess_input)


# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders

train_generator = data_generator.flow_from_directory(
        directory= 'drive/MyDrive/Modified Images Part 4',
        shuffle = True,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode ='categorical',
        subset = "training")

validation_generator = data_generator.flow_from_directory(
        directory= 'drive/MyDrive/Modified Images Part 4',
        shuffle = True,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical',
        subset= 'validation') 




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3415 images belonging to 4 classes.
Found 850 images belonging to 4 classes.


In [ ]:
test_generator = data_generator.flow_from_directory(
        directory= 'drive/MyDrive/DOP_test',
        shuffle = False,
        target_size=(image_size, image_size),
        batch_size=1,
        class_mode ='categorical'
        )



Found 234 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
# # we use SGD with a low learning rate
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(restnet)
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_dim=256))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer=Nadam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

'''resnet = ResNet50(weights='imagenet', include_top=False)
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,kernel_regularizer=regularizers.l2(0.001),activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=predictions)

# # we use SGD with a low learning rate
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import Adam


history = model.fit_generator(train_generator,
                    steps_per_epoch = train_generator.n// 16,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // 16,
                    epochs=50
                    )

model.save('model_trained_3class.hdf5')'''

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 131072)            23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33554688  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 1028      
Total params: 57,143,428
Trainable params: 33,555,716
Non-trainable params: 23,587,712
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


"resnet = ResNet50(weights='imagenet', include_top=False)\nx = resnet.output\nx = GlobalAveragePooling2D()(x)\nx = Dense(256,kernel_regularizer=regularizers.l2(0.001),activation='relu')(x)\nx = Dropout(0.5)(x)\n\npredictions = Dense(4, activation='softmax')(x)\n\nmodel = Model(inputs=resnet.input, outputs=predictions)\n\n# # we use SGD with a low learning rate\nfrom tensorflow.keras.optimizers import Nadam\nfrom tensorflow.keras.optimizers import Adam\n\n\nhistory = model.fit_generator(train_generator,\n                    steps_per_epoch = train_generator.n// 16,\n                    validation_data=validation_generator,\n                    validation_steps=validation_generator.n // 16,\n                    epochs=50\n                    )\n\nmodel.save('model_trained_3class.hdf5')"

In [ ]:

history = model.fit_generator(train_generator,
                    steps_per_epoch = train_generator.n// 16,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // 16,
                    epochs=50)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
213/213 [==============================] - 1348s 6s/step - loss: 1.3629 - accuracy: 0.4907 - val_loss: 0.9315 - val_accuracy: 0.7052
Epoch 2/50
213/213 [==============================] - 91s 427ms/step - loss: 1.1216 - accuracy: 0.5781 - val_loss: 0.8701 - val_accuracy: 0.7170
Epoch 3/50
213/213 [==============================] - 91s 427ms/step - loss: 0.9550 - accuracy: 0.6228 - val_loss: 0.8022 - val_accuracy: 0.7559
Epoch 4/50
213/213 [==============================] - 91s 427ms/step - loss: 0.9105 - accuracy: 0.6455 - val_loss: 0.7482 - val_accuracy: 0.7524
Epoch 5/50
213/213 [==============================] - 91s 425ms/step - loss: 0.8816 - accuracy: 0.6537 - val_loss: 0.7231 - val_accuracy: 0.7583
Epoch 6/50
213/213 [==============================] - 91s 426ms/step - loss: 0.8686 - accuracy: 0.6611 - val_loss: 0.7552 - val_accuracy: 0.7465
Epoch 7/50
213/213 [==============================] - 91s 425ms/step - loss: 0.8383 - accuracy: 0.6758 - val_loss: 0.6995 - val_acc

In [ ]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()


In [ ]:
plot_accuracy(history,'RESNET')
plot_loss(history,'RESNET')

In [ ]:
evaluate = model.evaluate_generator(test_generator, steps = test_generator.n//1, verbose = 1)